# Register Client and Create Access Token Notebook
- Find detailed information about client registration and access tokens in this blog post: [Authentication to SAS Viya: a couple of approaches](https://blogs.sas.com/content/sgf/2021/09/24/authentication-to-sas-viya/)
- Use the client_id to create an access token you can use in the Jupyter environment or externally for API calls to SAS Viya.
- You must add the following info to the script: client_id (you create), client_secret (you create), and, baseURL (SAS Viya URL)
- Client registration is a one time, admin task.
- Additional access token information is found at the end of this notebook.


### Run the cells below and follow the resulting instructions.

# Import packages and create variables

In [ ]:
import requests
import json
import os
import base64

# PLEASE SET OR CREATE the following variables BEFORE YOU EXECUTE THE FOLLOWING CELLS
# The client_id and client_secret values are up to you
# The client is either an application or acts on behalf of a user when making API calls
client_id=""     # api.client
client_secret="" # api.secret
baseURL = ""     # https://sasserver.sas.com

# Get register access token

In [ ]:
# create URL for access token for client registration
codeURL = baseURL + "/SASLogon/oauth/authorize?client_id=sas.cli&response_type=token"

# promt with instructions and entry access token URL
print(f"* Please visit the following site {codeURL} in an Incognito browser window")
print("* If provided a login prompt, add your SAS login credentials")
print("* Once authenticated, you'll be redirected to a new, lengthy URL that includes the access token")
print("* Copy the URL, and we'll parse it in the next step to extract the access token")
code = input("Please enter the URL you generated through the previous instructions, and then press Enter: ")

#process results and extract the access token
code2 = code.partition("&access_token=")[2]
register_access_token = code2.partition("&expires_in")[0]
print("The access token for client registration is: " + register_access_token)

# Register the client

In [ ]:
# create API call payload data
payload='{"client_id": "' + client_id +'","client_secret": "'+ client_secret +'","scope": ["openid", "uaa.user"],"authorized_grant_types": ["authorization_code","refresh_token"],"redirect_uri": "urn:ietf:wg:oauth:2.0:oob","access_token_validity": "42199", "autoapprove":true}'

# generate API call for register access token
url = f"{baseURL}/SASLogon/oauth/clients"
headers = {
'Content-Type': 'application/json',
'Authorization': "Bearer " + register_access_token
}

# process the results
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access token

In [ ]:
# create authorization url
codeURL = baseURL + "/SASLogon/oauth/authorize?client_id=" + client_id + "&response_type=code"

# enccode client string
client_string = client_id + ":" + client_secret
message_bytes = client_string.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

# promt with instructions and entry for auth code
print(f"* Please visit the following site {codeURL} in an Incognito browser window")
print("* If provided a login prompt, add your SAS login credentials with SASAdministrator rights")
print("* Once authenticated, you'll be redirected to an authoriztion screen, check all of the boxes that appear")
print("* This will result in a short string of numbers and letters such as `VAxVFVEnKr`; this is your authorization code; copy the code")
code = input("Please enter the authoriztion code you generated through the previous instructions, and then press Enter: ")

# generate API call for access token
url = f"{baseURL}/SASLogon/oauth/token#authorization_code"
payload = "grant_type=authorization_code&code=" + code
headers = {
'Accept': 'application/json',
'Content-Type': 'application/x-www-form-urlencoded',
'Authorization': "Basic " + base64_message
}

# process the results
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
access_token = json.loads(response.text)['access_token']
refresh_token = json.loads(response.text)['refresh_token']
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access_token.txt file 
directory = os.getcwd()
with open(directory + '/access_token.txt', 'w') as f:
    f.write(access_token)
print('The access token was stored for you as ' + directory + '/access_token.txt')

# Create refresh_token.txt file 
directory = os.getcwd()
with open(directory + '/refresh_token.txt', 'w') as f:
    f.write(refresh_token)
print('The refresh token was stored for you as ' + directory + '/refresh_token.txt')

## Notes on the access token
- The access token has a 12 hour time-to-live (ttl) by default.
- Use the refresh token to generate a new access token.
- The refressh token has a 90 day ttl.
- For the SAS Hackathon, once you've run through these steps once, you may use the getAccessToken_fromRefreshToken notebook to generate a fresh access token.
- The authorization code is good for 30 minutes and is only good for a single use. 
- You can generate a new authorization code by reusing the authorization URL.
- The access_token is valid in this Notebook and is transferable to other notebooks and used for external API calls.

# Use the refresh token to generate a new access token

In [ ]:
url = f"{baseURL}/SASLogon/oauth/token#refresh_token"

payload = "grant_type=refresh_token&refresh_token=" + refresh_token
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'Authorization': "Basic " + base64_message
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

print(response.text)
# process the results
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
access_token = json.loads(response.text)['access_token']
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access_token.txt file 
directory = os.getcwd()
with open(directory + '/access_token.txt', 'w') as f:
    f.write(access_token)
print('The access token was stored for you as ' + directory + '/access_token.txt')

# Test connection to SAS Viya via SWAT
This is an optional step that tests your access token ability to connect to SAS Viya

In [ ]:
import swat
conn = swat.CAS(f"{baseURL}/cas-shared-default-http", username=None, password=access_token)

In [ ]:
conn